In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
path = '/Users/higuchiitto/pyworks/研究室/deep_learning_seminer/simdata/'

train = pd.read_csv(path + 'saturn_data_train.csv', header=None)
test = pd.read_csv(path + 'saturn_data_eval.csv', header=None)

train_data = np.array(train.iloc[:, 1:])
train_label = np.array(train.iloc[:, 0]).astype(int)

test_data = np.array(test.iloc[:, 1:])
test_label = np.array(test.iloc[:, 0]).astype(int)

In [3]:
BATCH_SIZE = 100
log_dir = '../log'

In [4]:
#　Heの初期値
initializer = tf.random_normal_initializer(mean=0,
                                                                       stddev=(2/BATCH_SIZE)**0.5)

def inference(data_ph):
    with tf.variable_scope('fc1') as scope:
        weights = tf.get_variable(name='weights',
                                                     shape=(2, 32),
                                                     initializer=initializer)#.initialized_value()
        biases = tf.get_variable(name='biases',
                                                   shape=(32),
                                                   initializer=tf.zeros_initializer())#.initialized_value()
        product = tf.matmul(data_ph, weights)
        fc1 = tf.nn.relu(tf.nn.bias_add(product, biases))
        
    with tf.variable_scope('fc2') as scope:
        weights = tf.get_variable(name='weights',
                                                    shape=(32, 2),
                                                    initializer=initializer)#.initialized_value()
        biases = tf.get_variable(name='biases',
                                                   shape=(2),
                                                   initializer=tf.zeros_initializer())#.initialized_value()
        product = tf.matmul(fc1, weights)
        logits = tf.nn.relu(tf.nn.bias_add(product, biases))
        
    return logits

In [5]:
def _loss(logits, labels):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, name='xentropy_per_sample')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='xentropy')
    return cross_entropy_mean

In [6]:
def train(loss, global_step):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    gradients = optimizer.compute_gradients(loss)
    train_op = optimizer.apply_gradients(gradients, global_step=global_step)
    return train_op

In [7]:
global_step = tf.Variable(0, trainable=False)

train_loss = tf.Variable(0.0, trainable=False, name='train_loss')
train_precision = tf.Variable(0.0, trainable=False, name='train_precision')
test_loss = tf.Variable(0.0, trainable=False, name='test_loss')
test_precision = tf.Variable(0.0, trainable=False, name='test_precision')

train_loss_summary = tf.summary.scalar('loss/train', train_loss)
train_precision_summary = tf.summary.scalar('precision/train', train_precision)
test_loss_summary = tf.summary.scalar('loss/test', test_loss)
test_precision_summary = tf.summary.scalar('precision/test', test_precision)
    
data_ph = tf.placeholder(tf.float32, shape=(None, 2), name='input_placeholder')
label_ph = tf.placeholder(tf.int32, shape=(None), name='label_placeholder')

logits = inference(data_ph)
loss = _loss(logits, label_ph)
train_op = train(loss, global_step)
eval_correct = tf.nn.in_top_k(logits, label_ph, 1)

init = tf.global_variables_initializer()

end_flag = False

with tf.Session() as sess:    
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
    for step in range(100):
        shuffle = np.random.permutation(500)
        for i in range(5):
            start = BATCH_SIZE*i
            end = BATCH_SIZE*(i+1)
            
            shuffled_data = train_data[start:end]
            shuffled_label = train_label[start:end]
            
            _, loss_value, correction = sess.run([train_op, loss, eval_correct], feed_dict={data_ph : shuffled_data,
                                                                                            label_ph : shuffled_label})

            if i == 4:
                sess.run(train_loss.assign(loss_value))
                sess.run(train_precision.assign(correction.sum()/100))
                
                summary_str_1, summary_str_2= sess.run([train_loss_summary, train_precision_summary])
                summary_writer.add_summary(summary_str_1, step)
                summary_writer.add_summary(summary_str_2, step)
                
                print('Step {0}:\n trainning_precision: {1}'.format(step+1,correction.sum()/100))
                
                loss_value, test_correction = sess.run([loss, eval_correct], 
                                                                         feed_dict={data_ph : test_data, label_ph : test_label})
                sess.run(test_loss.assign(loss_value))
                sess.run(test_precision.assign(test_correction.sum()/100))
                
                summary_str_3, summary_str_4 = sess.run([test_loss_summary, test_precision_summary])
                summary_writer.add_summary(summary_str_3, step)
                summary_writer.add_summary(summary_str_4, step)
                
                print(' test_precision: {}'.format(test_correction.sum()/100))
                
                if test_correction.sum() == 100:
                    print('Perfect precision was completed!!')
                    end_flag = True
            if end_flag == True:
                break
        if end_flag == True:
            summary_writer.close()
            break

Step 1:
 trainning_precision: 0.64
 test_precision: 0.61
Step 2:
 trainning_precision: 0.73
 test_precision: 0.71
Step 3:
 trainning_precision: 0.81
 test_precision: 0.75
Step 4:
 trainning_precision: 0.83
 test_precision: 0.85
Step 5:
 trainning_precision: 0.91
 test_precision: 0.9
Step 6:
 trainning_precision: 0.94
 test_precision: 0.94
Step 7:
 trainning_precision: 0.99
 test_precision: 0.97
Step 8:
 trainning_precision: 0.99
 test_precision: 0.97
Step 9:
 trainning_precision: 1.0
 test_precision: 1.0
Perfect precision was completed!!
